## Charney-Eliassen model 

J. G. Charney & A. Eliassen (1949) A Numerical Method for Predicting the Perturbations of the Middle Latitude Westerlies, Tellus, 1:2, 38-54, DOI: 10.3402/tellusa.v1i2.8500
https://www.tandfonline.com/doi/abs/10.3402/tellusa.v1i2.8500

$
\frac{D}{D t}\frac{(\zeta + f)}{H} = 0,
$

$D/Dt = (\partial/\partial t + v \cdot \nabla)$

Assuming a low amplitude topographic forcing ($h_T \ll H$) and linearising around a time-independent quasi-geostrophic zonal mean flow ($u = [u] + u$, $v = v^\ast$ where $v^\ast \sim u^\ast \ll [u]$, here [·] and (∗) denotes zonal mean and zonal perturbation quantities, respectively) at a midlatitude $\beta$ plane ($f = f_0 + \beta y$), we can define the perturbation velocity field from a geostrophic stream function as ($u^\ast, v^\ast) = \hat{k} \times \nabla \psi^\ast$ and thus $\zeta^\ast = \nabla^2 \psi^\ast$. The resulting equation has wave solutions of the form $(\psi^\ast,h_T) = Re(\psi_0, h_0) \, exp\,[i(kx +ly)]$ and the complex amplitude of the height anomaly is <br>
$
\psi_0 = \frac{f_0}{H} \frac{h_0}{K^2 - K_s^2 - iR},
$

where $K = \sqrt{k^2 = l^2}$ is the total wavenumber, and the stationary wave number is defined as $K_s \equiv \sqrt{\beta/[u]}$. A linear damping term ($R = r\,K^2/k[u]$) is added to ensure bounded solutions as the height anomaly ($\psi_0$) resonates for a total wave number equal to the stationary wave number.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from scipy import fft, ifft

from netCDF4 import Dataset

### Define latitude

In [ ]:
latd = 45.
latr = np.deg2rad(latd)

### Domain and planetary constants

Gravitational acceleration: <br>
$g = 9.8 m\,s^{-1}$

Depth of atmosphere (m): <br>
$H = 8\cdot10^3$

Angular speed of rotation of the Earth (23 hours 56 minutes 4 seconds, or 86,164 s in a day): <br>
$\Omega = \frac{2 \pi}{86,164}$

Coriolis parameter: <br>
$f = 2 \, \Omega \, sin(\phi)$

Meridional gradient of the Coriolis parameter: <br>
$\beta = \frac{\partial f}{\partial y} = \frac{2 \, \Omega}{a} \, cos(\phi)$ <br>

Earth's radius (a): <br> 
$a \approx 6.371\cdot10^6$m

Earth's circumference: <br>
$x = 2 \, \pi \, a \, cos(\phi)$ 

Number of grid cells: <br>
n = 480 

In [ ]:
#Number of grid boxes
n = 480

## Gravitational acceleration (m/s^2)
g = 9.8

# Depth of atmosphere (m)
H = 8.e3

## Earth's radius (m)
a = 6.371e6

## Angular velocity (1/s)
omega = 2.*np.pi/86164.

## Coriolis parameter
f0 = 2*omega*np.sin(latr)

## Meridional gradient of Coriolis parameter
beta = 2.*omega*np.cos(latr)/a

## Earth's circumference at selected latitude
Lx = 2.*np.pi*a*np.cos(latr)

## Grid spacing 
dx = Lx/n

## Grid from -x_max/2:x_max/2-dx
x = np.arange(-Lx/2., Lx/2., dx)
x_degrees = x*360./Lx

## Utility array
ns = np.arange(n)

### Model parameters

Zonal wind (m/s): <br>
$u = 25$

Depth of atmosphere H(m): <br>
$H = 8\cdot10^3$

Damping timescale of 3 days (in seconds): <br>
$\tau = 5*86164$


Stationary wavenumber: <br>
$K^2_s = \beta\,/\,u$


Total wavenumber: <br>
$K^2 = k^2 + m^2$



In [ ]:
# Zonal wind (m/s)
u = 25.

## Meridional wave number:
m = 2.*np.pi/8.e6

# Zonal wave number
k = 2.*np.pi/Lx           

## Stationary wavenumber squared
Ks2 = beta/u

# Total wavenumber squared
K2 = (k*ns)**2. + m**2. 

# Damping time (seconds, 86400 seconds = 1 day)
tau = 5.*86164.

## Friction parameter
r = 1./tau # Inverse friction time
eps = ns*0.
eps[1:n] = r*K2[1:n]/(k*ns[1:n]*u)

### Gaussian topography

In [ ]:
def gaussian_topography():
    ## Mountain height h0(m):
    h0 = 4.e3

    ## Mountain width dx_h(m):
    dx_h = 1.e6 

    ## A is strength of topographic forcing: h_b~A\delta(x)
    A = h0*dx_h

    ## Define topography
    h = A*np.exp(-(x)**2/dx_h**2)/(np.pi**(0.5)*dx_h)
    return h

### Real world topography

In [ ]:
def world_topography():
    ds = Dataset('./ERAInt.surf_geopot.0.75x0.75.nc','r')
    iy = np.abs(ds.variables['latitude'][:]-latd).argmin()
    h = ds.variables['srfgeo'][0,iy-5:iy+5,:].mean(axis=0)/g
    return h
    

### Analytic solution to barotropic vorticity equation

In [ ]:
#h = gaussian_topography()
h = world_topography()

## Fourier transform topography
sh = fft(h)  # Fourier coef. (topography)
sh[0] = 0.   # Remove zonal mean

spsi = f0*sh/(H*(K2 - Ks2 - 1j*eps))

psi = np.real(ifft(spsi))

### Plot results

In [ ]:
plt.clf()

if latd == 0.: hem = ''
elif latd > 0.: hem = 'North'
else: hem = 'South'

ax = plt.subplot(2,1,1)
plt.plot(x_degrees, h, 'k', 
         label = 'Topography \n{}$^\circ$ {}'.format(int(latd),hem))
plt.ylabel('Topography (m)')
plt.ylim([0, 2500])
plt.xlim([-180, 180])
plt.xticks(np.arange(-180,210,30))
ax.legend(bbox_to_anchor=(1.35, 1.05))


ax = plt.subplot(2,1,2)
plt.plot(x_degrees, psi*f0/g, 'k',
         label = 'CE 1949')
plt.ylim([-250, 250])
plt.xlim([-180, 180])
plt.xticks(np.arange(-180,181,30))
plt.ylabel('Geopot. height (m)')


## ERA-Interim
ds = Dataset('./ERAInt.500hPa_geopot.djf.0.75x0.75.nc','r')
iy = np.abs(ds.variables['latitude'][:]-latd).argmin()
z = ds.variables['z'][0,0,iy-5:iy+5,:].mean(axis=0)/g
z -= np.mean(z,axis=0)
plt.plot(x_degrees, z, 'b', label = 'ERA-Interim')

ax.legend(bbox_to_anchor=(1.34, 1.05))
